In [1]:
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adadelta, Adam
from keras.preprocessing.image import ImageDataGenerator
from typing import Tuple, List, Dict
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

In [2]:
label_map={"0":"ANGRY","1":"HAPPY","2":"SAD","3":"SURPRISE","4":"NEUTRAL"}
data = pd.read_csv("fer2013.csv")
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


The data is divided into three subsets: Training, PublicTest (validation)& PrivateTest

In [3]:
def parse_df(source_df):
    X = [np.fromstring(i, dtype=np.int16, sep=' ') for i in source_df["pixels"]]
    y = source_df.values[:, :1]
    y = y.reshape(-1,)
    y = y.astype("int")
    return X, y

train_df = data[data["Usage"]=="Training"]
val_df = data[data["Usage"]=="PublicTest"]
test_df = data[data["Usage"]=="PrivateTest"]

features_train, targets_train = parse_df(train_df)
features_val, targets_val = parse_df(val_df)
features_test, targets_test = parse_df(test_df)

print(np.asarray(features_train).shape)
print(np.asarray(features_val).shape)
print(np.asanyarray(features_test).shape)
targets_train.reshape(-1,)

(28709, 2304)
(3589, 2304)
(3589, 2304)


array([0, 0, 2, ..., 4, 0, 4])

In [4]:
INPUT_SHAPE = (48, 48)
INPUT_DIM = INPUT_SHAPE[0]

In [6]:
def reshape_img_input(x: np.ndarray,
                      img_rows: int = 28,
                      img_cols: int = 28,
                      normalize: bool = True) -> Tuple:
  
    
    print(f'Original shape of nd-array: {x.shape}')
    
    if K.image_data_format() == 'channels_first':
        
        # the channel dimension goes to the front
        x1 = x.reshape(x.shape[0], 1, img_rows, img_cols)

        input_shape = (1, img_rows, img_cols)

    else:
        x1 = x.reshape(x.shape[0], img_rows, img_cols, 1)

        input_shape = (img_rows, img_cols, 1)

    x1 = x1.astype('float32')

    if normalize:
        x1 /= 255.
    
    
    return x1, input_shape

In [278]:
X_test, input_shape = reshape_img_input(x=X_test,
                                         normalize=True,
                                         img_rows=48, 
                                         img_cols=48)

Original shape of nd-array: (1800, 48, 48, 1)


In [7]:
X = np.array(features_train)

In [8]:
y = np.array(targets_train)

In [9]:
y = y.ravel()

In [10]:
X, input_shape = reshape_img_input(x=X,
                                         normalize=True,
                                         img_rows=48, 
                                         img_cols=48)

Original shape of nd-array: (28709, 2304)


In [ ]:
X_test_data, input_shape = reshape_img_input(x=X_test_data,
                                         normalize=True,
                                         img_rows=48, 
                                         img_cols=48)

In [279]:
X_test_data = X_test.reshape(X_test.shape[0], 1, 48, 48).astype('float32')

In [293]:
X_train_data, input_shape = reshape_img_input(x=X_train,
                                        normalize=True,
                                         img_rows=48, 
                                         img_cols=48)

Original shape of nd-array: (1800, 1, 48, 48)


In [294]:
X_train_data = X_train.reshape(X_train_data.shape[0], 1, 48, 48).astype('float32')

# Building the Model

In [12]:
model = Sequential()  
model.add(Conv2D(48, kernel_size=(3, 3), activation='relu', input_shape=(X.shape[1:]), padding='same'))  
model.add(Conv2D(48,kernel_size= (3, 3), activation='relu', padding='same'))    
model.add(MaxPooling2D(pool_size=(1,1)))  
model.add(Dropout(0.3))  
#2nd convolution layer  
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))  
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))    
model.add(MaxPooling2D(pool_size=(1,1)))  
model.add(Dropout(0.3))  
#3rd convolution layer  
model.add(Conv2D(124, (3, 3), activation='relu', padding='same'))  
model.add(Conv2D(124, (3, 3), activation='relu', padding='same'))    
model.add(MaxPooling2D(pool_size=(1,1)))  
model.add(Flatten())  
model.add(Dense(1,activation='softmax'))  
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 48)        480       
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 48)        20784     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 48)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 48, 48, 48)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 48, 48, 64)        27712     
                                                                 
 conv2d_3 (Conv2D)           (None, 48, 48, 64)        36928     
                                                        

In [13]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [14]:
model.fit(
    X,
    y,
    batch_size = 500,
    epochs = 5,
    verbose = 1)

Epoch 1/5


KeyboardInterrupt: 

# Building the second model

In [15]:
model2 = Sequential()  
model2.add(Conv2D(48, kernel_size=(4, 4), activation='relu', input_shape=(X.shape[1:]), padding='same'))  
model2.add(Conv2D(48,kernel_size= (4, 4), activation='relu', padding='same'))    
model2.add(MaxPooling2D(pool_size=(1,1)))  
model2.add(Dropout(0.7))  
#2nd convolution layer  
model2.add(Conv2D(64, (4, 4), activation='relu', padding='same'))  
model2.add(Conv2D(64, (4, 4), activation='relu', padding='same'))    
model2.add(MaxPooling2D(pool_size=(1,1)))  
model2.add(Dropout(0.1))  
#3rd convolution layer  
model2.add(Conv2D(128, (3, 3), activation='relu', padding='same'))  
model2.add(Conv2D(128, (3, 3), activation='relu', padding='same'))    
model2.add(MaxPooling2D(pool_size=(1,1)))  
model2.add(Flatten())  
model2.add(Dense(1,activation='softmax'))  
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 48, 48, 48)        816       
                                                                 
 conv2d_7 (Conv2D)           (None, 48, 48, 48)        36912     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 48, 48, 48)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 48, 48, 48)        0         
                                                                 
 conv2d_8 (Conv2D)           (None, 48, 48, 64)        49216     
                                                                 
 conv2d_9 (Conv2D)           (None, 48, 48, 64)        65600     
                                                      

In [16]:
model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [17]:
model2.fit(X, y, epochs = 5)

Epoch 1/5
898/898 [==============================] - 2038s 2s/step - loss: nan - accuracy: 0.0729
Epoch 2/5
898/898 [==============================] - 2380s 3s/step - loss: nan - accuracy: 0.1392
Epoch 3/5
898/898 [==============================] - 2213s 2s/step - loss: nan - accuracy: 0.1392
Epoch 4/5
199/898 [=====>........................] - ETA: 26:02 - loss: nan - accuracy: 0.1354

KeyboardInterrupt: 

# Evaluation

In [18]:
X_test = np.array(features_test)

In [19]:
y_test = np.array(targets_test)

In [20]:
y_test = y_test.ravel()

In [ ]:
score1 = model.evaluate(X_test,
                        y_test,
                        verbose=1)

print(f'Test loss: {score1[0]}:')
print('Test accuracy: {:.3f} %'.format(100 * score1[1]))

In [ ]:
score2 = model2.evaluate(X_test,
                        y_test,
                        verbose=1)

print(f'Test loss: {score2[0]}:')
print('Test accuracy: {:.3f} %'.format(100 * score2[1]))